In [ ]:
import random
import math
import pandas as pd

In [ ]:

# =======================
# DATA BAHAN PAKAN
# =======================

foods = [
    # name, category, protein_g, fiber_g, ca_mg, p_mg, kcal, sugar_g, mass_g per portion
    ("Jangkrik", "protein", 1.2, 0.05, 5, 20, 6, 0.0, 5),
    ("Ulat Hongkong", "protein", 1.5, 0.08, 10, 25, 7, 0.0, 5),
    ("Ayam Rebus", "protein", 3.0, 0.0, 6, 40, 20, 0.0, 10),
    ("Telur Rebus", "protein", 2.5, 0.0, 20, 90, 15, 0.0, 10),
    ("HPW Diet", "protein", 1.0, 0.3, 40, 20, 8, 0.3, 3),

    ("Apel", "fruit", 0.05, 0.4, 5, 10, 3, 1.25, 5),
    ("Pepaya", "fruit", 0.08, 0.45, 8, 12, 3, 1.0, 5),
    ("Melon", "fruit", 0.06, 0.2, 6, 9, 2.5, 0.75, 5),
    ("Pir", "fruit", 0.05, 0.42, 5, 11, 3, 1.25, 5),

    ("Wortel", "veg", 0.1, 0.6, 15, 10, 2, 0.6, 5),
    ("Buncis", "veg", 0.25, 0.5, 20, 12, 3, 0.3, 5),
    ("Jagung", "veg", 0.35, 0.2, 2, 15, 4, 0.5, 5),
    ("Selada", "veg", 0.08, 0.5, 15, 6, 1.5, 0.1, 5),

    ("Kalsium Bubuk", "supp", 0.0, 0.0, 500, 0, 0, 0.0, 0.5),
]

protein_idx = [i for i,f in enumerate(foods) if f[1]=="protein"]
fruit_idx   = [i for i,f in enumerate(foods) if f[1]=="fruit"]
veg_idx     = [i for i,f in enumerate(foods) if f[1]=="veg"]
supp_idx    = [i for i,f in enumerate(foods) if f[1]=="supp"]




In [ ]:

# =======================
# INDIVIDU (7 hari)
# Representasi: list[ 7 hari ], tiap hari = (protein, fruit, veg, supplement yes/no)
# =======================

def random_day():
    return [
        random.choice(protein_idx),
        random.choice(fruit_idx),
        random.choice(veg_idx),
        random.choice([0,1])  # supplement
    ]

def random_individual():
    return [random_day() for _ in range(7)]


In [ ]:


# =======================
# HITUNG NUTRISI PER HARI
# =======================

def compute_day_nutrition(day):
    prot, fru, veg, sup = day
    items = [foods[prot], foods[fru], foods[veg]]
    if sup==1:
        items.append(foods[supp_idx[0]])

    protein = sum(i[2] for i in items)
    fiber   = sum(i[3] for i in items)
    ca      = sum(i[4] for i in items)
    phos    = sum(i[5] for i in items)
    kcal    = sum(i[6] for i in items)
    sugar   = sum(i[7] for i in items)
    mass    = sum(i[8] for i in items)

    return protein, fiber, ca, phos, kcal, sugar, mass



In [ ]:

# =======================
# FITNESS
# =======================

def fitness(ind):
    total = [compute_day_nutrition(d) for d in ind]
    total_p = sum(x[0] for x in total)
    total_f = sum(x[1] for x in total)
    total_ca = sum(x[2] for x in total)
    total_ph = sum(x[3] for x in total)
    total_kcal = sum(x[4] for x in total)
    total_sugar = sum(x[5] for x in total)
    total_mass = sum(x[6] for x in total)

    if total_kcal == 0 or total_ph == 0:
        return 0

    protein_pct = (4*total_p)/total_kcal*100
    fiber_pct = (total_f/total_mass)*100
    cap_ratio = total_ca/total_ph
    sugar_pct = (4*total_sugar)/total_kcal*100

    score_p = max(0, 1 - abs(protein_pct - 30)/15)
    score_f = max(0, 1 - abs(fiber_pct - 35)/20)
    score_ca = math.exp(-((cap_ratio-2)**2)/(2*0.5**2))
    score_s = 1 if sugar_pct <= 10 else max(0, 1-(sugar_pct-10)/10)
    score_k = 1 if 50 <= total_kcal <= 80 else max(0, 1 - abs(total_kcal-65)/65)

    prot_sources = [d[0] for d in ind]
    variety = len(set(prot_sources))/len(ind)
    score_v = variety

    return (0.25*score_p + 0.20*score_f + 0.25*score_ca +
            0.10*score_s + 0.10*score_k + 0.10*score_v)



In [ ]:

# =======================
# GENETIC ALGORITHM
# =======================

def crossover(a,b):
    point = random.randint(1,6)
    return a[:point] + b[point:], b[:point] + a[point:]

def mutate(ind, rate=0.05):
    for d in ind:
        if random.random()<rate: d[0]=random.choice(protein_idx)
        if random.random()<rate: d[1]=random.choice(fruit_idx)
        if random.random()<rate: d[2]=random.choice(veg_idx)
        if random.random()<rate: d[3]=random.choice([0,1])

def tournament(pop):
    k = random.sample(pop,3)
    return max(k, key=fitness)

POP = 50
GEN = 120

pop = [random_individual() for _ in range(POP)]

for _ in range(GEN):
    new = [max(pop, key=fitness)]  # elitisme
    while len(new)<POP:
        p1 = tournament(pop)
        p2 = tournament(pop)
        c1, c2 = crossover(p1,p2)
        mutate(c1); mutate(c2)
        new.append(c1); new.append(c2)
    pop = new




In [ ]:

# =======================
# TAMPILKAN HASIL
# =======================
best = max(pop, key=fitness)
result = []
days = ["Senin","Selasa","Rabu","Kamis","Jumat","Sabtu","Minggu"]

for i,day in enumerate(best):
    prot, fru, veg, sup = day
    result.append([
        days[i],
        foods[prot][0],
        foods[fru][0],
        foods[veg][0],
        "Ya" if sup==1 else "Tidak"
    ])

df = pd.DataFrame(result, columns=["Hari","Protein","Buah","Sayur","Kalsium?"])
df
